## SVM
RBF Kernel, C=100, gamma=0.01

In [1]:
import time
import math 
import random
import numpy as np
import pandas as pd
from cvxopt import solvers
solvers.options['show_progress'] = False
from cvxopt import matrix

from sklearn.metrics import precision_score

from sklearn.metrics import recall_score

In [2]:
ker_linear = lambda X1, X2: X1.dot(X2.T)

def kernel_rbf(X1, X2, gamma):
    if len(X1.shape) == 1:
        X1 = X1.reshape(1, -1) # (N1, d)
    if len(X2.shape) == 1:
        X2 = X2.reshape(1, -1) # (N2, d)
    X1 = np.expand_dims(X1, axis=1) # (N1, 1, d)
    X2 = np.expand_dims(X2, axis=0) # (1, N2, d)
    
    # broadcasting trick
    return np.exp(-gamma * np.sum((X1 - X2) ** 2, axis=2))

def train_kernel_svm(X, y, k, C, gamma):
    """
    inputs
    X: data matrix, shape (N, d)
    y: label matrix, shape (N,)
    k: if k is None, then compute the kernel by XX^T; else k could be a function or precomputed matrix
    C: coefficient of slack terms in primal optimization, scalar 
    
    returns
    w: weight, shape (N,)
    b: bias, scalar
    """
    start_time = time.time()

    N = len(y)
    if callable(k):
        # print("kernel!")
        K = k(X, X, gamma)
    elif k is None:
        k = ker_linear
        K = X.dot(X.T)
    else:
        K = k
    # prints(K)

    P = K.reshape(N, N) * y.reshape(N, 1).dot(y.reshape(1, N))
    q = -np.ones(N)

    G = np.concatenate((np.eye(N), -np.eye(N)))
    # h = np.concatenate((C * np.ones(N), np.zeros(N)))
    
    C_pos = (np.count_nonzero(y == 1)/N)*C  
    C_neg = (np.count_nonzero(y == -1)/N)*C 
    # print(C_pos, C_neg)

    y_C = np.zeros(N)
    pos = np.where(y == 1)
    neg = np.where(y == -1)
    y_C[pos] = C_pos 
    y_C[neg] = C_neg 
    h = np.concatenate((y_C, np.zeros(N)))
    
    A = y.reshape(1, N)
    A = A.astype('float')
    b = np.zeros(1)
    
    sol = solvers.qp(matrix(P), matrix(q), matrix(G), matrix(h), matrix(A), matrix(b))
    # print("done")
    alpha = np.array(sol['x'])
    # print(alpha)
    alpha[alpha < 1e-4] = 0
    alpha = alpha.reshape(-1)
    
    # is_support_vector = (0 < alpha) & (alpha < C)

    is_support_vector = []
    for i in range(len(alpha)):
        value = False
        if alpha[i] > 0:
            if y[i] == -1:
                value = abs(alpha[i] - C_neg) <= 0.01
            elif y[i] == 1:
                value = abs(alpha[i] - C_pos) <= 0.01 
        is_support_vector.append(value)

    y_sv = y[is_support_vector]
    X_sv = X[is_support_vector]
    b = (y_sv - ((alpha * y).reshape(-1, 1)*k(X, X_sv, gamma)).sum(axis=0)).mean()

    end_time = time.time()
    print("SVM training time:", end_time - start_time, "s")
    return alpha, b

In [3]:
def get_pred_kernel_svm(alpha, b, X, y, ker, gamma=1):
    return lambda x: (alpha * y * ker(X, x, gamma).reshape(-1)).sum(axis=0) + b

def svm_score(alpha, b, X_train, y_train, X, y, ker, gamma=1):
    pred_kernel_svm = get_pred_kernel_svm(alpha, b, X_train, y_train, ker, gamma)
    
    start_time = time.time()
    preds = np.array([pred_kernel_svm(x) for x in X])

    score = (preds * y > 0).mean()
    print(score)
    print(preds)
    precision = precision_score(y, np.sign(preds), average='binary')
    recall = recall_score(y, np.sign(preds), average='binary')
    
    end_time = time.time() 
    print("SVM eval time:", end_time - start_time, "s")
    return score, precision, recall

## KNN 
Chi-squared distance
K=71
Features=cwt energy bands, std, energy, max

In [4]:
def chi_squared_distance(A, B):
    return 0.5* np.sum(((A-B)**2) / (A+B), axis=0)

class KNN:
    def __init__(self, K, distance_metric, trainX, trainY):
        """
        Parameters:
            K is an int representing the number of closest neighbors to consider
            distance_metric is one of euclidean or manhattan
            trainX is d by n array
            trainY is 1 by n array
        """
        self.trainX = trainX
        self.trainY = trainY
        self.pos_size = np.count_nonzero(self.trainY == 1)
        self.neg_size = np.count_nonzero(self.trainY == -1)
        self.K = K
        self.metric = distance_metric

    def calc_distances(self, testX):
        dists = np.vstack([self.metric(self.trainX, testX[:, i:i+1]) for i in range(testX.shape[1])])
        return dists
        
    def find_top_neighbor_labels(self, dists):
        """
        Parameters:
            dists is  m x n array D where D[i, j] is the distance between test sample i and train sample j
        Returns:
            an m x K array L where L[i, j] is the label of the jth closest neighbor to test sample i
            in case of ties, the neighbor which appears first in the training set is chosen
        """
        neighbors = np.argsort(dists)[:, :self.K]
        neighbor_labels = self.trainY.squeeze()[neighbors]
        return neighbor_labels

    def find_top_neighbor(self, dists):
        top_distances = np.sort(dists)[:, :self.K]
        top_labels = self.trainY.squeeze()[np.argsort(dists)[:, :self.K]]
        return top_distances, top_labels
    
    def predict(self, testX):
        """
        Parameters:
            testX is d by m array
        Returns:
            predicted is 1 x m array P where P[0, i] is the predicted label for test sample i 
        """
        neighbor_labels = self.find_top_neighbor_labels(self.calc_distances(testX))
        neighbor_labels = neighbor_labels.astype(int)
        predicted = np.hstack([np.bincount(vals).argmax() for vals in neighbor_labels])
        return predicted
    
    def score(self, testX, testY):
        """
        Parameters:
            testX is d by m array of input data
            testY is 1 by m array of labels for the input data
        Returns:
            the accuracy of the KNN predictions across the test set
        """
        
        start_time = time.time()
        pred = self.predict(testX)
        # print("Precision:", precision_score(testY, pred, average='binary'))
        # print("Recall:", recall_score(testY, pred, average='binary'))
        precision = precision_score(testY, pred, average='binary')
        recall = recall_score(testY, pred, average='binary')
        score = np.sum(pred == testY) / len(testY)
        end_time = time.time()
        print("KNN eval time:", end_time - start_time, "s")
        return score, precision, recall


In [16]:
from imblearn.over_sampling import SMOTE

def train_knn(X, y, distance, k):
    start_time = time.time()
    oversample = SMOTE()
    x_tmp, y_tmp = oversample.fit_resample(X, y)
    classifier = KNN(k, distance, x_tmp.T, y_tmp)
    end_time = time.time()
    print("KNN training time:", end_time - start_time, "s") # should be fairly negligible?
    return classifier

## Random Forest 
100 trees, 10 samples per leaf

In [6]:
class Node:
    def __init__(self, feature, value, num_samples_seizure, num_samples_noseizure):
        self.feature = feature
        self.value = value
        self.num_samples_seizure = num_samples_seizure
        self.num_samples_noseizure = num_samples_noseizure
        self.left = None
        self.right = None

class RandomForest:
    def __init__(self, num_trees, min_samples_leaf, trainX, trainY, samples_per_tree=None, features_per_tree=None, features_to_train_on = None):
        '''
        samples_per_tree is the number of samples (with replacement) used to construct each tree
        features_per_tree is the number of features to consider when making splits in each tree
        features_to_train_on is a list of indices that correspond to features, if None: all features are used
        '''
        self.num_trees = num_trees
        self.min_samples_leaf = min_samples_leaf
        self.trainX = trainX
        self.trainY = trainY
        self.pos_size = np.count_nonzero(self.trainY == 1)
        self.neg_size = np.count_nonzero(self.trainY == -1)
        self.trees = []

        if samples_per_tree is None:
            self.samples_per_tree = trainX.shape[0]
        else:
            self.samples_per_tree = samples_per_tree

        if features_per_tree is None:
            self.features_per_tree = int(math.sqrt(trainX.shape[1]))
        else:
            self.features_per_tree = features_per_tree
        
        if features_to_train_on is None:
            self.features_to_train_on = np.arange(trainX.shape[1])
        else:
            self.features_to_train_on = features_to_train_on
    
    def build_tree(self, data_indices, features, weights):
        best_feature, best_val, best_left, best_right = self.best_split(data_indices, features, weights)
        if best_feature is None:
            return None
        root = Node(best_feature, best_val, np.count_nonzero(self.trainY[data_indices] == 1), len(data_indices) - np.count_nonzero(self.trainY[data_indices] == 1))
        root.left = self.build_tree(best_left, features, weights)
        root.right = self.build_tree(best_right, features, weights)
        return root

    def train(self):
        self.trees = []
        for _ in range(self.num_trees):
            #randomly sample from X with replacement
            samples_indices = np.random.choice(self.trainX.shape[0], size=self.samples_per_tree, replace=True)

            #randomly choose features to consider
            features = np.random.choice(self.features_to_train_on, size=self.features_per_tree, replace=False)

            positive_class_count = np.count_nonzero(self.trainY[samples_indices] == 1)
            negative_class_count = self.samples_per_tree - positive_class_count

            tree = self.build_tree(samples_indices, features, (1.0 / positive_class_count, 1.0 / negative_class_count))

            self.trees.append(tree)

    def predict(self, testX):
        '''
        label of 1 = seizure, label of 0 = no seizure
        '''
        predictions = np.zeros(testX.shape[0])
        for i in range(testX.shape[0]):
            predict_proba = self.predict_proba_ensemble(testX[i])
            if predict_proba[0] > predict_proba[1]:
                predictions[i] = 1
            else:
                predictions[i] = -1
        return predictions

    def predict_proba_ensemble(self, x):
        '''
        returns mean (prob seizure, prob no seizure)
        '''
        probs_seizure = 0
        probs_noseizure = 0
        for tree in self.trees:
            (prob_seizure, prob_noseizure) = self.predict_proba(x, tree)
            probs_seizure += prob_seizure
            probs_noseizure += prob_noseizure
        return (probs_seizure, probs_noseizure)

    def predict_proba(self, x, tree):
        '''
        x is a row of data
        returns (prob of seizure, prob of not seizure)
        '''
        #seizure_weight = 1.0 / tree.num_samples_seizure
        #noseizure_weight = 1.0 / tree.num_samples_noseizure
        while tree is not None:
            if x[tree.feature] <= tree.value:
                if tree.left is None:
                    return (float(tree.num_samples_seizure) / (tree.num_samples_seizure + tree.num_samples_noseizure), float(tree.num_samples_noseizure) / (tree.num_samples_seizure + tree.num_samples_noseizure))
                tree = tree.left
            else:
                if tree.right is None:
                    return (float(tree.num_samples_seizure) / (tree.num_samples_seizure + tree.num_samples_noseizure), float(tree.num_samples_noseizure) / (tree.num_samples_seizure + tree.num_samples_noseizure))
                tree = tree.right

    def perform_split(self, data, feature, split_val):
        left = []
        right = []
        for row_index in data:
            if self.trainX[row_index][feature] <= split_val:
                left.append(row_index)
            else:
                right.append(row_index)
        return left, right
    
    def best_split(self, data, features, weights):
        best_score = 2**31-1
        best_feature, best_val, best_left, best_right = None, None, None, None
        for index in features:
            feature, val, left, right, score = self.find_split(data, index, weights)
            if score < best_score:
                best_score = score
                best_feature, best_val, best_left, best_right = feature, val, left, right
        return best_feature, best_val, best_left, best_right

    def find_split(self, data, feature, weights):
        best_score = 2**31-1
        left = None
        right = None
        val = None
        for row_index in data:
            split_val = self.trainX[row_index][feature]
            left_temp, right_temp = self.perform_split(data, feature, split_val)
            if len(left_temp) < self.min_samples_leaf or len(right_temp) < self.min_samples_leaf:
                continue
            split_gini_score = self.gini_score([left_temp, right_temp], weights)
            if split_gini_score < best_score:
                best_score = split_gini_score
                left = left_temp
                right = right_temp
                val = split_val
        return feature, val, left, right, best_score

    def gini_score(self, splits, weights):
        '''
        splits is list of lists where each list has indices into trainX of rows in that group
        '''
        gini = 0
        all_samples = [index for split in splits for index in split]
        positive_class_weight = weights[0]
        negative_class_weight = weights[1]
        t_p = positive_class_weight * np.count_nonzero(self.trainY[all_samples] == 1) + negative_class_weight * np.count_nonzero(self.trainY[all_samples] == -1)
        for split in splits:
            if len(split) == 0:
                continue
            split_score = 0
            positive_class_count = np.count_nonzero(self.trainY[split] == 1) #seizure
            negative_class_count = np.count_nonzero(self.trainY[split] == -1) 
            t_c = positive_class_weight * positive_class_count + negative_class_weight * negative_class_count
            split_score += (positive_class_weight * positive_class_count/t_c) ** 2
            split_score += (negative_class_weight * negative_class_count/t_c) ** 2
            gini += t_c/t_p * (1.0 - split_score) 
        return gini

    def score(self, testX, testY):
        start_time = time.time()
        pred = self.predict(testX)
        score = np.sum(pred == testY) / testX.shape[0]
        precision = precision_score(testY, pred, average='binary')
        recall = recall_score(testY, pred, average='binary')
        end_time = time.time()
        print("Random Forest eval time:", end_time - start_time, "s")
        return score, precision, recall

In [21]:
def train_rfc(X, y, num_tree, num_sample):
    start_time = time.time()
    classifier = RandomForest(num_tree, num_sample, X, y, samples_per_tree=500)
    classifier.train()
    end_time = time.time()
    print("Random Forest training time:", end_time - start_time, "s") # should be fairly negligible?
    return classifier

## Ensemble

In [8]:
# hyperparameters 
# SVM
C = 1
gamma = 0.01 
kernel = kernel_rbf

# KNN
features = [14, 13, 4, 7, 12, 1, 15, 10] 
k = 71
distance = chi_squared_distance

# Random Forest 
num_trees = 100 
num_samples_per_leaf = 10

In [9]:
train = pd.read_csv('data/train_data.csv').values
X_train = train[:, :-1].copy()
X_train_knn = train[:, features].copy()
y_train = train[:, -1].copy()
y_train_knn = train[:, -1].copy()
y_train_knn[y_train_knn==-1] = 0

validate = pd.read_csv('data/val_data.csv').values
X_val = validate[:, :-1].copy()
X_val_knn = validate[:, features].copy()
y_val = validate[:, -1].copy()
y_val_knn = validate[:, -1].copy()
y_val_knn[y_val_knn==-1] = 0

test = pd.read_csv('data/test_data.csv').values
X_test = test[:, :-1].copy()
X_test_knn = test[:, features].copy()
y_test = test[:, -1].copy()
y_test_knn = test[:, -1].copy()
y_test_knn[y_test_knn==-1] = 0

In [10]:
print(X_train, y_train)
alpha, b = train_kernel_svm(X_train, y_train, kernel, C, gamma)
print(alpha, b)

[[-0.40193103  0.3601071   0.01985798 ...  4.778956    4.76422315
   4.72137961]
 [-0.18775274  0.19391389  0.0204222  ...  4.35321124  4.40797066
   4.46735753]
 [-0.25534373  0.20556815 -0.01132332 ...  4.6581769   4.73820826
   4.7669041 ]
 ...
 [-0.19410745  0.20757138  0.03035962 ...  4.41947911  4.42540977
   4.44277172]
 [-0.2780837   0.37422512 -0.00802251 ...  4.56386628  4.63457401
   4.64847662]
 [-0.18197574  0.25089026  0.02713012 ...  4.60340105  4.70450984
   4.757542  ]] [ 1. -1. -1. ... -1.  1. -1.]
SVM training time: 1108.2112007141113 s
[0. 0. 0. ... 0. 0. 0.] 1.7632260883553508


In [11]:
print(alpha, b)
score, _, _ = svm_score(alpha, b, X_train, y_train, X_train, y_train, kernel, gamma)
print("SVM train:", score)

score, _, _ = svm_score(alpha, b, X_train, y_train, X_val, y_val, kernel, gamma)
print("SVM val:", score)

[0. 0. 0. ... 0. 0. 0.] 1.7632260883553508
0.970105446244157
[ 1.99836787 -0.31524533 -0.29591247 ... -0.3660442   1.8863911
 -0.30773197]
SVM eval time: 23.696786165237427 s
SVM train: 0.970105446244157
0.9539130434782609
[-0.30715638 -0.30663783  1.763227   ... -0.40105281 -0.30068552
 -0.15634671]
SVM eval time: 2.7591071128845215 s
SVM val: 0.9539130434782609


In [18]:
knn = train_knn(X_train_knn, y_train_knn, distance, k)

KNN training time: 0.022972822189331055 s


In [19]:
score, _, _ = knn.score(X_train_knn.T, y_train_knn)
print("KNN train:", score)

KNN eval time: 22.549869060516357 s
KNN train: 0.9613001413197086


In [22]:
random_forest = train_rfc(X_train, y_train, num_trees, num_samples_per_leaf)

Random Forest training time: 397.1856937408447 s


In [23]:
score, _, _ = random_forest.score(X_train, y_train)
print("Random Forest train:", score)

Random Forest eval time: 3.045464515686035 s
Random Forest train: 0.9630394608109577


In [35]:
def svm_predict(X, y):
    predictor_svm = get_pred_kernel_svm(alpha, b, X, y, kernel, gamma)
    return np.array([predictor_svm(x) for x in X])

def knn_predict(X, y):
    pred = knn.predict(X.T)
    pred[pred==0] = -1
    return knn.predict(X.T)

def rfc_predict(X, y):
    return random_forest.predict(X)

In [48]:
# basic voting ensemble classifier:
def pred_basic_ensemble(X, y, X_knn, y_knn):
    from scipy import stats

    svm_pred = svm_predict(X, y)
    knn_pred = knn_predict(X_knn, y_knn)
    rfc_pred = rfc_predict(X, y)
    all_preds = np.array([svm_pred, knn_pred, rfc_pred])
    
    final_preds = stats.mode(all_preds, axis=0)[0].flatten()
    return final_preds

def score_basic_ensemble(X, y, X_knn, y_knn):
    preds = pred_basic_ensemble(X, y, X_knn, y_knn)
    for i in preds.shape[0]:
        if preds[i] != 1 and preds[i] != -1:
            print(preds[i])
    score = np.sum(preds == y) / len(y)
    precision = precision_score(y, preds, average='binary')
    recall = recall_score(y, pred, average='binary')

    return score, precision, recall

In [49]:
score, precision, recall = score_basic_ensemble(X_train, y_train, X_train_knn, y_train_knn)
print("train error:", 1 - score)

score, precision, recall = score_basic_ensemble(X_val, y_val, X_val_knn, y_val_knn)
print("val error:", 1 - score)

score, precision, recall = score_basic_ensemble(X_test, y_test, X_test_knn, y_test_knn)
print("test error:", 1 - score)
print("test precision:", precision)
print("test recall:", recall)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [26]:
# linear svm for stacking ensemble
def train_svm(X, y, C=1, print_matrix=False):
    """
    inputs
    X: data matrix, shape (N, d)
    y: label matrix, shape (N,)
    C: coefficient of slack terms in primal optimization, scalar 
    
    returns
    w: weight, shape (N,)
    b: bias, scalar
    """
    
    N = X.shape[0]
    Xy = X * y.reshape(N, 1)
    P = Xy.dot(Xy.T)
    q = -np.ones(N)

    G = np.concatenate((np.eye(N), -np.eye(N)))
    h = np.concatenate((C * np.ones(N), np.zeros(N)))

    A = y.reshape(1, N)
    b = np.zeros(1)

    sol = solvers.qp(matrix(P), matrix(q), matrix(G), matrix(h), matrix(A), matrix(b))
    alphas = np.array(sol['x'])
    alphas[alphas < 1e-4] = 0
    
    w = (alphas.reshape(N, 1) * y.reshape(N, 1) * X).sum(axis=0)
    b = (y - X.dot(w))[(0 < alphas.reshape(N)) & (alphas.reshape(N) < C - 1e-4)].mean()
    return w, b

def get_pred_svm(w, b):
    return lambda x: w.dot(x) + b

In [27]:
# stacking ensemble
# use linear SVM for meta-learning
def train_stacking_ensemble(X, y, X_knn, y_knn, C=1):
    svm_pred = svm_predict(X, y)
    knn_pred = knn_predict(X_knn, y_knn)
    rfc_pred = rfc_predict(X, y)
    all_preds = np.array([svm_pred, knn_pred, rfc_pred])
    
    svm_input = all_preds.T
    return train_svm(svm_input, y, C) # use linear kernel by default 

def pred_stacking_ensemble(X, y, X_knn, y_knn, w, w0):
    print(w, w0)

    svm_pred = svm_predict(X, y)
    knn_pred = knn_predict(X_knn, y_knn)
    rfc_pred = rfc_predict(X, y)
    all_preds = np.array([svm_pred, knn_pred, rfc_pred])

    lin_svm_pred = get_pred_svm(w, w0)
    return np.array([lin_svm_pred(x) for x in X])

def score_stacking_ensemble(X, y, X_knn, y_knn, w, w0):
    preds = pred_stacking_ensemble(X, y, X_knn, y_knn, w, w0)

    score = np.sum(preds == y) / len(y)
    precision = precision_score(y, preds, average='binary')
    recall = recall_score(y, preds, average='binary')

    return score, precision, recall

In [28]:
C_ensemble = [0.01, 0.1, 1, 10, 100]

for C_val in C_ensemble:
    print("C =", C_val)
    w, w0 = train_stacking_ensemble(X_train, y_train, X_train_knn, y_train_knn, C_val)
    
    score, precision, recall = score_stacking_ensemble(X_train, y_train, X_train_knn, y_train_knn, w, w0)
    print("train error:", 1 - score)

    score, precision, recall = score_stacking_ensemble(X_val, y_val, X_val_knn, y_val_knn, w, w0)
    print("val error:", 1 - score)

    score, precision, recall = score_stacking_ensemble(X_test, y_test, X_test_knn, y_test_knn, w, w0)
    print("test error:", 1 - score)
    print("test precision:", precision)
    print("test recall:", recall)

    print("\n")

C = 0.01


NameError: name 'pred_kernel_svm' is not defined